## Follow up with Azure OpenAI the course Building Systems with the ChatGPT API

### L4: Process Inputs: Chain of Thought Reasoning

This notebook is based on the course: [Process Inputs: Chain of Thought Reasoning](https://learn.deeplearning.ai/chatgpt-building-system/lesson/5/chain-of-thought-reasoning) 


In [1]:
import os
import openai # this will require pip install openai
import tiktoken # this will require pip install tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AZURE_OPENAI_KEY')
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = "https://cog-moww6huavlklg.openai.azure.com/"


In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo",
                                 deployment="chat",
                                 printResponse=False,
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        engine=deployment,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    if printResponse:
        print(response)
    return response.choices[0].message["content"]

## Chain-of-Thought Prompting

In [3]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product category doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 5 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step and include response to user.
"""

In [4]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a question about two specific products, the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### Both products are available in the store. The price of the BlueWave Chromebook is $249.99 and the price of the TechPro Desktop is $999.99.
Step 3:#### The user did not make any assumptions in their message.
Step 4:#### The price difference between the BlueWave Chromebook and the TechPro Desktop is $750.00.
Step 5:#### The BlueWave Chromebook is $750.00 cheaper than the TechPro Desktop. 
Response to user:#### The BlueWave Chromebook is $750.00 cheaper than the TechPro Desktop. The price of the BlueWave Chromebook is $249.99 and the price of the TechPro Desktop is $999.99.


## Variaciones:
Uno de los problemas que enfrentamos con Azure Search como datasource es que no funciona tan bien system_message (o todavía no he entendido como usarlo bien). Según la documentación: [Use your data: AI Search system messagee](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/use-your-data?tabs=ai-search#system-message) 

Por tanto en las siguientes pruebas voy ir intercalando las instrucciones en el rol para entender como se comporta y si hay aparentes variaciones

#### Moviendo las instrucciones de system message al rol de "user"

In [5]:
delimiter = "####"
customer_query = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""


user_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product category doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 5 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step and include response to user.

The customer query: {delimiter}{customer_query}{delimiter}
"""

In [6]:

messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between two specific products.

Step 2:#### The two products mentioned are the BlueWave Chromebook and the TechPro Desktop.

Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.

Step 4:#### The assumption is incorrect. The BlueWave Chromebook is actually less expensive than the TechPro Desktop.

Step 5:#### The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99. Therefore, the BlueWave Chromebook is $750 less expensive than the TechPro Desktop.

Response to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99. Therefore, the BlueWave Chromebook is $750 less expensive than the TechPro Desktop.


Aparentemente no hubo mucha diferencia entre las respuestas.

La siguiente prueba es para ver que pasa si no especifico todos los pasos en el formato de la respuesta:

In [7]:
delimiter = "####"
customer_query = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""


user_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product category doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step and include response to user.

The customer query: {delimiter}{customer_query}{delimiter}
"""

In [9]:
messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = get_completion_from_messages(messages, printResponse=True)
print(response)

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1:#### \nThe user is asking about the price difference between the BlueWave Chromebook and the TechPro Desktop.\n\nStep 2:#### \nThe BlueWave Chromebook is priced at $249.99, and the TechPro Desktop is priced at $999.99.\n\nStep 5:#### \nResponse to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99.",
        "role": "assistant"
      }
    }
  ],
  "created":

Parece ser que no todos los pasos se incluyeron en la respuesta, ahora procederemos a
### Mover system message al rol de "assistant"

In [11]:
delimiter = "####"
assistant_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product category doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Step 5:{delimiter} <step 5 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step and include response to user.
"""

In [12]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'assistant', 
 'content': assistant_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between two specific products.
Step 2:#### The two products in question are the BlueWave Chromebook and the TechPro Desktop. The prices of these products are as follows:
- BlueWave Chromebook: $249.99
- TechPro Desktop: $999.99
Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption made by the user is incorrect. The TechPro Desktop is actually more expensive than the BlueWave Chromebook.
Step 5:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The price difference between the two products is $750.00. 
Response to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The price difference between the two products is $750.00.


In [5]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the employee queries.
The employee query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide if employee provides a name \ 
if name is not provided then you must ask for his name.
Step 2:{delimiter} Second decide if employee provides years working in the company \
if years working in the company are not provided then you must ask for years in the company. 

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Response to employee:{delimiter} <response to employee>

Make sure to include {delimiter} to separate every step and include response to employee.
"""


In [6]:
user_message = f"""
Cuáles son los beneficios de Baufest"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The employee has not provided their name.
Step 2:#### The employee has not provided the number of years they have been working at Baufest.

Response to employee:#### Hola, gracias por tu pregunta. ¿Podrías proporcionarme tu nombre y cuántos años llevas trabajando en Baufest? 

En cuanto a los beneficios de trabajar en Baufest, podemos mencionar que ofrecemos un ambiente de trabajo colaborativo y dinámico, oportunidades de crecimiento y desarrollo profesional, programas de capacitación y entrenamiento, flexibilidad horaria, días de home office, seguro médico, entre otros. ¿Te gustaría saber más sobre algún beneficio en particular?


In [11]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking if the store sells TVs.
Step 2:#### The user is not asking about a specific product.
Step 3:#### N/A
Step 4:#### N/A
Step 5:#### Yes, we do sell TVs. We have a variety of brands and sizes available. Please let us know if you have any specific requirements or preferences.


In [12]:
user_message = f"""
¿Usted vende tacos?"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is not asking about a specific product.
Step 2:#### N/A
Step 3:#### N/A
Step 4:#### N/A
Step 5:#### We apologize, but we do not sell tacos. Our store specializes in computers and laptops. Is there anything else we can assist you with?


In [13]:
user_message = f"""
Which is your cheaper laptop?"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about a specific product category, i.e. laptops.
Step 2:#### Yes, we have 5 laptops available. The cheapest laptop we have is the BlueWave Chromebook priced at $249.99.
Step 3:#### N/A
Step 4:#### N/A
Step 5:#### Our cheapest laptop is the BlueWave Chromebook priced at $249.99. It is a compact and affordable Chromebook for everyday tasks.


## Inner Monologue
- Since we asked the LLM to separate its reasoning steps by a delimiter, we can hide the chain-of-thought reasoning from the final output that the user sees.

In [15]:
def remove_inner_monologue(delimiter, response):
    try:
        final_response = response.split(delimiter)[-1].strip()
    except Exception as e:
        final_response = "Sorry, I'm having trouble right now, please try asking another question."
    return final_response


In [16]:
print(remove_inner_monologue(delimiter, response))

Our cheapest laptop is the BlueWave Chromebook priced at $249.99. It is a compact and affordable Chromebook for everyday tasks.
